In [1]:
import pandas as pd
import numpy as np

In [2]:
!head -n 3 ./data/covtype.data

2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [3]:
cover = pd.read_csv("./data/covtype.data", sep=",", header=None)

In [4]:
cover.shape[0]
cover.shape[1]

55

In [5]:
classes = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Rawah', 'Neota', 'Comanche_Peak', 'Cache_la_Poudre', 'ELU_2702', 'ELU_2703', 'ELU_2704', 'ELU_2705', 'ELU_2706', 'ELU_2717', 'ELU_3501', 'ELU_3502', 'ELU_4201', 'ELU_4703', 'ELU_4704', 'ELU_4744', 'ELU_4758', 'ELU_5101', 'ELU_5151', 'ELU_6101', 'ELU_6102', 'ELU_6731', 'ELU_7101', 'ELU_7102', 'ELU_7103', 'ELU_7201', 'ELU_7202', 'ELU_7700', 'ELU_7701', 'ELU_7702', 'ELU_7709', 'ELU_7710', 'ELU_7745', 'ELU_7746', 'ELU_7755', 'ELU_7756', 'ELU_7757', 'ELU_7790', 'ELU_8703', 'ELU_8707', 'ELU_8708', 'ELU_8771', 'ELU_8772', 'ELU_8776', 'Cover_Type']

In [6]:
len(classes)

55

In [7]:
cover.columns = classes

In [8]:
cover.iloc[:,:10]

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,2596,51,3,258,0,510,221,232,148,6279
1,2590,56,2,212,-6,390,220,235,151,6225
2,2804,139,9,268,65,3180,234,238,135,6121
3,2785,155,18,242,118,3090,238,238,122,6211
4,2595,45,2,153,-1,391,220,234,150,6172
...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837
581008,2391,152,19,67,12,95,240,237,119,845
581009,2386,159,17,60,7,90,236,241,130,854
581010,2384,170,15,60,5,90,230,245,143,864


In [9]:
cover['Cover_Type'].value_counts(normalize=True).sort_index()

1    0.364605
2    0.487599
3    0.061537
4    0.004728
5    0.016339
6    0.029891
7    0.035300
Name: Cover_Type, dtype: float64

In [10]:
split = int((581012/100)*70)
train = pd.concat([cover[cover.Cover_Type==1].sample(int(split*0.364605)), cover[cover.Cover_Type==2].sample(int(split*0.487599)), cover[cover.Cover_Type==3].sample(int(split*0.061537)), cover[cover.Cover_Type==4].sample(int(split*0.004728)), cover[cover.Cover_Type==5].sample(int(split*0.016339)), cover[cover.Cover_Type==6].sample(int(split*0.029891)), cover[cover.Cover_Type==7].sample(int(split*0.035300))])
test = cover.loc[~cover.index.isin(train.index)]

stratified sampling using the normalized value_counts of each Cover_Type. 70-30 split between train and test.

In [11]:
contiProbList = []   #continuous variables' probability density
priorProbList = []   #prior probability
condProbList = []    #conditional probability

In [12]:
def probDensityFunc(coverType):
    mean = train.iloc[:,:10][train['Cover_Type'] == coverType].mean()
    var = train.iloc[:,:10][train['Cover_Type'] == coverType].var()
    std = train.iloc[:,:10][train['Cover_Type'] == coverType].std()
    train.loc[train['Cover_Type'] == coverType, :'Horizontal_Distance_To_Fire_Points'] = (1/((np.sqrt(2*np.pi)) * std)) * np.exp(-((cover.iloc[:,:10]-mean)**2 / (2*var)))
    
    newList = train.loc[train['Cover_Type'] == coverType, :'Horizontal_Distance_To_Fire_Points'].mean().tolist()
    contiProbList.append(newList)
    
    mean = test.iloc[:,:10][test['Cover_Type'] == coverType].mean()
    var = test.iloc[:,:10][test['Cover_Type'] == coverType].var()
    std = test.iloc[:,:10][test['Cover_Type'] == coverType].std()
    test.loc[test['Cover_Type'] == coverType, :'Horizontal_Distance_To_Fire_Points'] = (1/((np.sqrt(2*np.pi)) * std)) * np.exp(-((cover.iloc[:,:10]-mean)**2 / (2*var)))   

convert continuous values into probability density function values per Cover_Type

In [13]:
for i in range(1, 8):
    probDensityFunc(i)

C:\Users\dnkr9\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


run method for all Cover_Types between 1 - 7

In [14]:
contiProbList

[[0.0018163047343661485,
  0.0022309512392392566,
  0.04181843397203185,
  0.0013068513176401296,
  0.005409717135842,
  0.00018278210885838608,
  0.011707024198045304,
  0.015950388602149777,
  0.007900909382366969,
  0.00023607568956416006],
 [0.001500804043962504,
  0.0024731641182945095,
  0.04016844203863223,
  0.001371133921146786,
  0.005338205083168606,
  0.00017113069694904806,
  0.011653064080377026,
  0.015536507628282969,
  0.007877500163093628,
  0.0002076841318370729],
 [0.0013961162386787007,
  0.002450181975648608,
  0.030487369102628767,
  0.0019501608102304344,
  0.0048649376394345745,
  0.0004662144553111505,
  0.006776242135277694,
  0.010084196925845891,
  0.0052504221314639355,
  0.000546250597056128],
 [0.0026923052808477156,
  0.0033836875430933505,
  0.02864372979963026,
  0.002050281805122638,
  0.00479266047202299,
  0.0007473063408451092,
  0.011832204215671828,
  0.013532203706750857,
  0.00555435622955784,
  0.0005646988199444891],
 [0.002994085205800733,


In [15]:
classes
binaryClasses = classes[10:54]

In [16]:
priorProb = train['Cover_Type'].value_counts(normalize=True).sort_index()
priorProb

1    0.364608
2    0.487604
3    0.061536
4    0.004726
5    0.016339
6    0.029889
7    0.035298
Name: Cover_Type, dtype: float64

In [17]:
priorProbList = priorProb.tolist()

In [18]:
priorProbList

[0.36460758833841894,
 0.48760397636604597,
 0.061536305362881516,
 0.004725807284431145,
 0.016338704164955753,
 0.02988913285616285,
 0.03529848562710381]

In [19]:
def condProbability(dataset, column, coverType):
    total = train.loc[train['Cover_Type'] == coverType, column].count()
    x = train.loc[(train['Cover_Type'] == coverType) & (train[column] == 1), column].count()
    if (x == 0 or x == total):
        x += 1
        total += 2
    x /= total
    return x

get positive counts for each feature per Cover_Type number.
in the event of zero-frequency occurences, add 1 count of '1' value and 1 count of '0' value to the total.

In [20]:
for i in range(1,8):
    coverList = []
    for p in binaryClasses:
        x = condProbability(cover, p, i)
        coverList.append(x)
    condProbList.append(coverList)

run method for all Cover_Types between 1 - 7

In [21]:
cover1 = np.concatenate((contiProbList[0], condProbList[0]))
cover2 = np.concatenate((contiProbList[1], condProbList[1]))
cover3 = np.concatenate((contiProbList[2], condProbList[2]))
cover4 = np.concatenate((contiProbList[3], condProbList[3]))
cover5 = np.concatenate((contiProbList[4], condProbList[4]))
cover6 = np.concatenate((contiProbList[5], condProbList[5]))
cover7 = np.concatenate((contiProbList[6], condProbList[6]))

join the continuous and categorical lists together by Cover_Type

In [22]:
cover1

array([1.81630473e-03, 2.23095124e-03, 4.18184340e-02, 1.30685132e-03,
       5.40971714e-03, 1.82782109e-04, 1.17070242e-02, 1.59503886e-02,
       7.90090938e-03, 2.36075690e-04, 4.99841524e-01, 8.77757322e-02,
       4.12382744e-01, 6.74358853e-06, 6.74358853e-06, 6.74358853e-06,
       6.74358853e-06, 8.63190974e-04, 6.74358853e-06, 6.74358853e-06,
       6.74358853e-06, 2.02310385e-04, 7.21573705e-04, 4.45082846e-03,
       3.58089381e-03, 1.29276336e-02, 1.02571365e-02, 6.74358853e-06,
       6.74358853e-06, 3.06837417e-03, 9.98064564e-04, 3.43927654e-04,
       1.18756196e-02, 1.72907942e-02, 3.78320419e-03, 1.21055790e-01,
       1.68531294e-01, 5.24321080e-02, 5.59725397e-04, 1.40268533e-03,
       2.91326954e-03, 2.22541423e-04, 1.98385563e-01, 3.63956382e-02,
       5.57365110e-02, 1.00784290e-01, 8.53008018e-02, 4.18108128e-04,
       4.18108128e-03, 4.72057564e-05, 6.74358853e-06, 4.14533978e-02,
       3.72992912e-02, 2.25171458e-02])

In [23]:
features = classes[:54]

In [24]:
likelihoods= pd.DataFrame({"features": features, "cover_type_1": cover1, "cover_type_2": cover2, "cover_type_3": cover3, "cover_type_4": cover4, "cover_type_5": cover5, "cover_type_6": cover6, "cover_type_7": cover7})

In [25]:
likelihoods
likelihoods.sort_values("cover_type_1", ascending=False).head(10)

,features,cover_type_1,cover_type_2,cover_type_3,cover_type_4,cover_type_5,cover_type_6,cover_type_7
10,Rawah,0.499842,0.515637,0.000040,0.00052,0.398947,0.000082,0.248328
12,Comanche_Peak,0.412383,0.441849,0.396012,0.00052,0.601053,0.440276,0.640777
42,ELU_7745,0.198386,0.251455,0.000040,0.00052,0.118736,0.000082,0.040401
36,ELU_7202,0.168531,0.073698,0.000040,0.00052,0.074342,0.001398,0.034202
35,ELU_7201,0.121056,0.026141,0.000040,0.00052,0.000150,0.000082,0.007384
45,ELU_7756,0.100784,0.104009,0.002877,0.00052,0.046050,0.011517,0.041934
11,Neota,0.087776,0.031925,0.000040,0.00052,0.000150,0.000082,0.110894
46,ELU_7757,0.085301,0.089249,0.000120,0.00052,0.056433,0.030684,0.029326
44,ELU_7755,0.055737,0.046896,0.000040,0.00052,0.032656,0.003784,0.009891
37,ELU_7700,0.052432,0.034446,0.000040,0.00052,0.007976,0.007897,0.009613


cover_type_1's best predictor is feature 'Rawah'

In [26]:
def getMax(a,b,c,d,e,f,g):
    
    if (a == max(a,b,c,d,e)):
        return 1
    elif (b == max(a,b,c,d,e)):
        return 2
    elif (c == max(a,b,c,d,e)):
        return 3
    elif (d == max(a,b,c,d,e)):
        return 4
    elif (e == max(a,b,c,d,e)):
        return 5
    elif (f == max(a,b,c,d,e)):
        return 6
    elif (g == max(a,b,c,d,e)):
        return 7       

In [27]:
def classifyNB0(vec2Classify, cover1, cover2, cover3, cover4, cover5, cover6, cover7, priorProb):

    c1 = np.sum(np.log(np.power(cover1, vec2Classify))) + np.log(priorProb[0])
    c2 = np.sum(np.log(np.power(cover2, vec2Classify))) + np.log(priorProb[1])
    c3 = np.sum(np.log(np.power(cover3, vec2Classify))) + np.log(priorProb[2])
    c4 = np.sum(np.log(np.power(cover4, vec2Classify))) + np.log(priorProb[3])
    c5 = np.sum(np.log(np.power(cover5, vec2Classify))) + np.log(priorProb[4])
    c6 = np.sum(np.log(np.power(cover6, vec2Classify))) + np.log(priorProb[5])
    c7 = np.sum(np.log(np.power(cover7, vec2Classify))) + np.log(priorProb[6])
    
    return getMax(c1, c2, c3, c4, c5, c6, c7)

use sum of logs instead of multiplication to negate numerical underflow

In [28]:
testMatrix = test.iloc[:,:54].to_numpy()
testMatrix.shape

(174309, 54)

In [29]:
testPred = [classifyNB0(testVec, cover1, cover2, cover3, cover4, cover5, cover6, cover7, priorProbList) for testVec in testMatrix]

In [30]:
test["predict"] = testPred

C:\Users\dnkr9\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
test.head(30)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,ELU_7757,ELU_7790,ELU_8703,ELU_8707,ELU_8708,ELU_8771,ELU_8772,ELU_8776,Cover_Type,predict
5,0.000400,0.003648,0.031909,0.001889,0.003961,0.000085,0.013011,0.017711,0.010975,7.257249e-06,...,0,0,0,0,0,0,0,0,2,2
7,0.000760,0.002645,0.015173,0.002204,0.005251,0.000290,0.017253,0.014726,0.007284,2.451032e-08,...,0,0,0,0,0,0,0,0,5,2
16,0.000833,0.001898,0.008159,0.001939,0.004688,0.000297,0.016494,0.011743,0.005945,4.219981e-08,...,0,0,0,0,0,0,0,0,5,2
22,0.003992,0.004137,0.035653,0.002203,0.005936,0.000061,0.007855,0.014855,0.004680,6.930134e-08,...,0,0,0,0,0,0,0,0,5,2
26,0.000044,0.004184,0.035272,0.001320,0.004470,0.000342,0.016494,0.010174,0.007220,4.235589e-07,...,0,0,0,0,0,0,0,0,5,2
28,0.001805,0.003665,0.011761,0.001803,0.006696,0.000184,0.015562,0.017094,0.010510,1.458161e-05,...,0,0,0,0,0,0,0,0,2,2
30,0.002863,0.004225,0.012804,0.001594,0.006407,0.000080,0.007019,0.012311,0.002941,3.780959e-08,...,0,0,0,0,0,0,0,0,5,2
31,0.000114,0.004005,0.024717,0.001939,0.004831,0.000295,0.015725,0.014455,0.007894,3.156560e-07,...,0,0,0,0,0,0,0,0,5,2
32,0.000105,0.003953,0.021318,0.001961,0.005044,0.000299,0.016494,0.013860,0.007657,3.271469e-07,...,0,0,0,0,0,0,0,0,5,2
38,0.003790,0.003879,0.012804,0.001961,0.006925,0.000064,0.007855,0.005485,0.001777,6.081329e-08,...,0,0,0,0,0,0,0,0,5,2


In [32]:
pd.crosstab(test['Cover_Type'], test['predict'], margins=True)

predict,1,2,3,4,5,All
Cover_Type,,,,,,
1,31098,32401,54,0,0,63553
2,12960,69517,2513,1,0,84991
3,0,903,9740,84,0,10727
4,0,0,645,180,0,825
5,241,2350,257,0,0,2848
6,54,1805,3287,65,0,5211
7,5215,902,20,0,17,6154
All,49568,107878,16516,330,17,174309


In [34]:
accuracy = (31098 + 69517 + 9740 + 180) / 174309
accuracy

0.6341324888559972

1 - 0.489%
2 - 0.818%
3 - 0.908%
4 - 0.218%
5 - 0%
6 - 0%
7 - 0%